In [1]:
import mysql.connector
import config
import yelp_key
import requests
import json

In [48]:
# Establish connection to database
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.passwd)

c = cnx.cursor()

In [3]:
key = yelp_key.key

## Business Search Endpoint

In [37]:
count = 0
#Business Search Endpoint
url = 'https://api.yelp.com/v3/businesses/search'

term = 'coffee'
location = "56 Bogart St, Brooklyn, NY 11206"
latitude = 40.710946
longitude = -73.951152
sort_by = 'distance'


headers = {
            'Authorization': 'Bearer {}'.format(key),
           }

url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50,
                'latitude' : 40.710946,
                'longitude' : -73.951152,
                'offset': count, 
                'radius' : 5500,
                'sort_by' : sort_by.replace(' ', '+')
          }

In [38]:
def business_search(url, headers, params):
    response = requests.get(url, headers = headers, params = url_params)
    business = json.loads(response.text)
    return business


In [20]:
business_data = business_search(url, headers = headers, params = url_params)

In [21]:
len(business_data['businesses'])

50

In [22]:
#how many cafes were returned?
business_data['total']

4800

There are 4800 cafe's total in response

In [9]:
#preview first result 
business_data['businesses'][0]

{'id': 'm-Syr8BpEbgK-6zSgY_7Kg',
 'alias': 'cafe-argentino-brooklyn',
 'name': 'Cafe Argentino',
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/ve7_oOWMKev1hmlxYlLxZg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/cafe-argentino-brooklyn?adjust_creative=d31wiC8swpcod_WEbvT7ew&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=d31wiC8swpcod_WEbvT7ew',
 'review_count': 250,
 'categories': [{'alias': 'argentine', 'title': 'Argentine'},
  {'alias': 'steak', 'title': 'Steakhouses'},
  {'alias': 'cafes', 'title': 'Cafes'}],
 'rating': 4.0,
 'coordinates': {'latitude': 40.711199, 'longitude': -73.95137},
 'transactions': ['delivery', 'restaurant_reservation', 'pickup'],
 'price': '$$',
 'location': {'address1': '499 Grand St',
  'address2': '',
  'address3': '',
  'city': 'Brooklyn',
  'zip_code': '11211',
  'country': 'US',
  'state': 'NY',
  'display_address': ['499 Grand St', 'Brooklyn, NY 11211']},
 'phone': '+17187829477',
 'display_phone': '(71

What Attributes are of interest from this endpoint?
- id
- name
- review count
- rating
- price
- location > address1 +
- city
- zip
- state
- phone

What might be collected from the business/details?

- id
- name
- review_count
- rating
- price
- location > address 1 + city + zip + state
- lat / long geocode
- phone
- categories

(basically all the same fields as business/search... but with geocode available


business/reviews endpoint:

for each review..

- id 
- rating
- user > id, name
- text


In [39]:
def parse_business(business_json):
    
    parsed_businesses = []
    
    for biz in business_json['businesses']:
        try:
            tup = (biz['id'], biz['name'], biz['rating'],len(biz['price']),
                   ' '.join(biz['location']['display_address']),
                   biz['display_phone'], biz['review_count'],
                   biz['categories'][0]['title'])
            parsed_businesses.append(tup)
        except:
            tup = (biz['id'], biz['name'], biz['rating'], 0,
                       ' '.join(biz['location']['display_address']),
                     biz['display_phone'], biz['review_count'],
                       biz['categories'][0]['title'])

            parsed_businesses.append(tup)
            
    return parsed_businesses
            

In [23]:
parsed_businesses = parse_business(business_data)

In [25]:
parsed_businesses[1]

('DO91C6baD0RSO-AN6cKCCA',
 'The West',
 4.0,
 2,
 '379 Union Ave Brooklyn, NY 11211',
 '(718) 599-1704',
 224,
 'Coffee & Tea')

In [40]:
def get_review_url(biz_data):
    review_url = []
    for i in range(len(biz_data)):
        review_url.append(f'https://api.yelp.com/v3/businesses/{biz_data[i][0]}/reviews')
    return review_url

In [15]:
review_url = get_review_url(biz_data)

In [41]:
def review_call(review_urls, headers):
    review_data = []
    for urls in review_urls:
        response = requests.get(urls, headers = headers)
        review = json.loads(response.text)
        review_data.append(review)
    return review_data

In [18]:
review_raw = review_call(review_url, headers)

In [19]:
review_raw[0]

{'reviews': [{'id': 'xUG_lVmFGTsqI2IafdilVg',
   'url': 'https://www.yelp.com/biz/cafe-argentino-brooklyn?adjust_creative=d31wiC8swpcod_WEbvT7ew&hrid=xUG_lVmFGTsqI2IafdilVg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=d31wiC8swpcod_WEbvT7ew',
   'text': 'It has been delicious before and it continues to provide comfort during these uncertain time. They have a rather plentiful takeout menu! From empanadas yum!...',
   'rating': 5,
   'time_created': '2020-06-21 18:43:35',
   'user': {'id': '4ecxgN6DXQSMSigQgkXIAA',
    'profile_url': 'https://www.yelp.com/user_details?userid=4ecxgN6DXQSMSigQgkXIAA',
    'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/dUJfIW3oXemLEyH6gaCZPw/o.jpg',
    'name': 'Daniela B.'}},
  {'id': 'hlaJQ7M4QMczimvtWMJwxQ',
   'url': 'https://www.yelp.com/biz/cafe-argentino-brooklyn?adjust_creative=d31wiC8swpcod_WEbvT7ew&hrid=hlaJQ7M4QMczimvtWMJwxQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=d31wiC8swpcod_WEbvT7

In [42]:
def get_business_ids(business_data):
    ids = []
    for i in range(len(business_data)):
        ids.append(business_data[i][0])
    return ids
        

In [23]:
business_ids = get_business_ids(biz_data)

In [43]:
def add_business_ids(business_ids, review_data):
    id_plus_reviews = []
    for i in range(len(review_data)):
        join = (business_ids[i], review_data[i])
        id_plus_reviews.append(list(join))
        
    join_dict = list(map(lambda x: {'business_id': x[0], 'reviews' : x[1]}, id_plus_reviews))
    return join_dict  

In [62]:
reviews_with_business_id = add_business_ids(business_ids, review_raw)

In [63]:
reviews_with_business_id[0]

{'business_id': 'WWvlUxNgmYaT_umfRU3cvA',
 'reviews': {'reviews': [{'id': 'w9IsUv18UsBCf1sDd0KTPQ',
    'url': 'https://www.yelp.com/biz/niforos-coffee-shop-long-island-city?adjust_creative=d31wiC8swpcod_WEbvT7ew&hrid=w9IsUv18UsBCf1sDd0KTPQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=d31wiC8swpcod_WEbvT7ew',
    'text': "So glad to welcome this space back to the neighborhood. New name, new management, similar menu (though I'm told now with more Mexican food), same pleasingly...",
    'rating': 5,
    'time_created': '2019-07-21 09:31:50',
    'user': {'id': '1Z-povofP-her5ElltKejQ',
     'profile_url': 'https://www.yelp.com/user_details?userid=1Z-povofP-her5ElltKejQ',
     'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/RXkWQYDdbeUY4uXVWEP_zw/o.jpg',
     'name': 'Carl B.'}},
   {'id': 'jq7XnsS6_Qw3XrV8F4QaYw',
    'url': 'https://www.yelp.com/biz/niforos-coffee-shop-long-island-city?adjust_creative=d31wiC8swpcod_WEbvT7ew&hrid=jq7XnsS6_Qw3XrV8F4QaYw&utm_

In [44]:
def parse_reviews(reviews_with_business_id):
    parsed_reviews = []
    
    for x in reviews_with_business_id:

        if 'error' in x['reviews'].keys():
            tup = (x['business_id'], x['reviews']['error']['code'])
            parsed_reviews.append(tup)
        else:
            try:
                for i in range(len(x['reviews'])):
                    tup = (x['business_id'], x['reviews']['reviews'][i]['id'],
                           x['reviews']['reviews'][i]['time_created'],
                           x['reviews']['reviews'][i]['rating'],
                           x['reviews']['reviews'][i]['user']['id'],
                           x['reviews']['reviews'][i]['user']['name'],
                           x['reviews']['reviews'][i]['text'])

                    parsed_reviews.append(tup)
                    i+1
            except Exception as e:
                print(e,': missing reviews')
    return parsed_reviews
            


In [64]:
parsed_reviews = parse_reviews(reviews_with_business_id)

list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews


In [65]:
parsed_reviews[0]

('WWvlUxNgmYaT_umfRU3cvA',
 'w9IsUv18UsBCf1sDd0KTPQ',
 '2019-07-21 09:31:50',
 5,
 '1Z-povofP-her5ElltKejQ',
 'Carl B.',
 "So glad to welcome this space back to the neighborhood. New name, new management, similar menu (though I'm told now with more Mexican food), same pleasingly...")

In [39]:
# c.execute('''CREATE DATABASE coffee IF NOT EXISTS''')

In [45]:
# c.execute('''DROP TABLE reviews;
#             DROP TABLE cafes;''')

In [49]:
c.execute('''USE coffee''')

In [55]:
c.execute('''CREATE TABLE cafes(

                ID VARCHAR(100) PRIMARY KEY,
                NAME VARCHAR(100),
                RATING FLOAT,
                PRICE INT,
                ADDRESS VARCHAR(160),
                PHONE VARCHAR(100),
                REVIEW_COUNT INT,
                CATEGORIES VARCHAR(200));'''
         )

In [56]:
c.execute(                     
            '''CREATE TABLE reviews(
            
                CAFE_ID VARCHAR(100),
                REVIEW_ID VARCHAR(100) PRIMARY KEY,
                REVIEW_DATE DATETIME,
                RATING FLOAT,
                USER_ID VARCHAR(100),
                USER_NAME VARCHAR(100),
                REVIEW VARCHAR(1000),
                CONSTRAINT FOREIGN KEY (CAFE_ID) REFERENCES cafes(ID)
                                    ); 
                
                                   '''
         )

In [52]:
def cafes_insert(c, cnx, parsed_business):
    
    c.executemany('''INSERT INTO coffee.cafes
                    (ID, NAME, RATING, PRICE, ADDRESS, PHONE, REVIEW_COUNT, CATEGORIES)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s);''', parsed_business)
    cnx.commit()

In [60]:
cafes_insert(c, cnx, parsed_businesses)

In [53]:
def review_insert(c, cnx, parsed_review):
    for x in parsed_review:
        if len(x) == 7:
            c.execute('''INSERT INTO coffee.reviews (CAFE_ID, REVIEW_ID, REVIEW_DATE, RATING, USER_ID, USER_NAME, REVIEW)
                        VALUES (%s, %s, %s, %s, %s, %s, %s)''', x)
        else:
            c.execute('''INSERT INTO coffee.reviews (CAFE_ID, REVIEW)
                        VALUES (%s, %s)''', x)
        
    cnx.commit()

In [61]:
review_insert(c, cnx, parsed_reviews)

In [57]:
count = 0 
#4800 cafes total. Yelp restricts business endpoint daily pull to 1000, throwing error if limit is reached
while count < 1000:
    url_params['offset'] = count
    business_data = business_search(url, headers = headers, params = url_params)
    parsed_businesses = parse_business(business_data)
    review_urls = get_review_url(parsed_businesses)
    review_raw = review_call(review_urls, headers)
    business_ids = get_business_ids(parsed_businesses)
    reviews_with_business_id = add_business_ids(business_ids, review_raw)
    parsed_reviews = parse_reviews(reviews_with_business_id)
    cafes_insert(c, cnx, parsed_businesses)
    review_insert(c, cnx, parsed_reviews)
    count += 50
    if count > 1000:
        break

list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing reviews
list index out of range : missing 

KeyError: 'businesses'